In [ ]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
from operator import itemgetter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.stem import WordNetLemmatizer
import re
# from xgboost import XGBClassifier

def readGz(path):
  for l in gzip.open(path, 'rt'):
    yield eval(l)

def readCSV(path):
  f = gzip.open(path, 'rt')
  f.readline()
  for l in f:
    yield l.strip().split(',')

In [ ]:
################## 1

In [ ]:
def accuracy(predictions, y):
  correct = 0

  # print(predictions[1] == y[1])

  for i in range(len(y)):
    if predictions[i] == y[i]:
      correct += 1

  # print(len(y))
  return correct / len(predictions)

In [ ]:
def Jaccard(s1, s2):
    s1 = set(s1)
    s2 = set(s2)
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [ ]:
### Would-read baseline: just rank which books are popular and which are not, and return '1' if a book is among the top-ranked
ratingsValid = []
ratingsPerItem = defaultdict(list)
ratingsPerUser = defaultdict(list)

bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
  ratingsPerUser[user].append(book)
  ratingsPerItem[book].append(user)
  ratingsValid.append((user,book))
  bookCount[book] += 1
  totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > 1.5 * totalRead/2: break
  # if count > totalRead/1.5: break

In [ ]:
def predict(u,b):
  maxSim = 0
  users = set(ratingsPerItem[b])
  for b2 in ratingsPerUser[u]:
      sim = Jaccard(users,set(ratingsPerItem[b2]))
      if sim > maxSim:
          maxSim = sim
  if maxSim > 0.035 or len(ratingsPerItem[b]) > 29:
      pred = 1
  else:
    pred = 0
  return pred

In [ ]:
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,b = l.strip().split(',')
  pred = predict(u,b)
  predictions.write(u + ',' + b + "," + str(pred) + "\n")
  # if b in return1 and pred == 1:
  #   predictions.write(u + ',' + b + ",1\n")
  # else:
  #   predictions.write(u + ',' + b + ",0\n")
predictions.close()

In [ ]:
###################### 2

In [ ]:
data = []

for d in readGz("train_Category.json.gz"):
    data.append(d)

In [ ]:
data[0]

{'user_id': 'u75242413',
 'review_id': 'r45843137',
 'rating': 4,
 'review_text': "a clever book with a deeply troubling premise and an intriguing protagonist. Thompson's clean, sparse prose style kept each page feeling light even as some rather heavy existential questions dropped upon them. I enjoyed it. \n and that cover design is boom-pow gorgeous.",
 'n_votes': 1,
 'genre': 'mystery_thriller_crime',
 'genreID': 3}

In [ ]:
counts = []
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
words = [x[1] for x in counts[:1000]]

In [ ]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [ ]:
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) # offset
    return feat

In [ ]:
nltk.download('stopwords')
useless = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
reviews = []

for i in range(len(data)):
    review = re.sub('[^a-zA-Z]', ' ', data[i]['review_text']).lower().split()
    review = ' '.join([lemmatizer.lemmatize(r) for r in review if not r in set(useless)])
    reviews.append(review)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews)
print(X.shape)
y = [d['genreID'] for d in data]

(100000, 216104)


In [ ]:
mod = linear_model.LogisticRegression(C=0.9)
mod.fit(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(C=0.9)

In [ ]:
input = []
for l in readGz("test_Category.json.gz"):
  input.append(l)

In [ ]:
reviews = []

for i in range(len(input)):
    review = re.sub('[^a-zA-Z]', ' ', input[i]['review_text']).lower().split()
    review = ' '.join([lemmatizer.lemmatize(r) for r in review if not r in set(useless)])
    reviews.append(review)

Xvalid = vectorizer.transform(reviews)
pred = mod.predict(Xvalid)

In [ ]:
### Category prediction baseline: Just consider some of the most common words from each category
i = 0
catDict = {
  "children": 0,
  "comics_graphic": 1,
  "fantasy_paranormal": 2,
  "mystery_thriller_crime": 3,
  "young_adult": 4
}

predictions = open("predictions_Category.csv", 'w')
predictions.write("userID,reviewID,prediction\n")
for l in readGz("test_Category.json.gz"):
  cat = pred[i]
  predictions.write(l['user_id'] + ',' + l['review_id'] + "," + str(cat) + "\n")
  i += 1
predictions.close()